In [111]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

ChromeDriverManager().install()
browser = webdriver.Chrome()


In [107]:

url = 'https://www.yes24.com/Product/Category/BestSeller?CategoryNumber=001&sumgb=06'
browser.get(url)

# from selenium.webdriver.common.by import By
# browser.find_element(By.CLASS_NAME, 'uU7dJb').text

In [39]:
# 한 개의 베스트 셀러 링크 수집
browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

# 1페이지의 베스트 셀러 전체 링크 수집
datas = browser.find_elements(By.CLASS_NAME, 'gd_name')

for i in datas:
    print(i.text)

하루 한 장 나의 어휘력을 위한 필사 노트
너에게 들려주는 단단한 말
불변의 법칙
2024 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 상
내 몸 혁명
그의 운명에 대한 아주 개인적인 생각
죽이고 싶은 아이 2
언젠가 우리가 같은 별을 바라본다면
2024 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 하
세이노의 가르침
ETS 토익 정기시험 기출문제집 1000 Vol. 4 RC
ETS 토익 정기시험 기출문제집 1000 Vol. 4 LC
신화의 숲
허송세월
THE MONEY BOOK 더 머니북
나를 소모하지 않는 현명한 태도에 관하여
나는 다정한 관찰자가 되기로 했다
초역 부처의 말
2024 큰별쌤 최태성의 별별한국사 기출 500제 한국사능력검정시험 심화 (1,2,3급)
모순
꽃길이 따로 있나, 내 삶이 꽃인 것을
마흔에 읽는 쇼펜하우어 (200쇄 기념 확장판)
돈의 심리학 (30만 부 기념 스페셜 에디션)
나는 메트로폴리탄 미술관의 경비원입니다


In [112]:
#3페이지까지의 데이터 제목 크롤링

browser = webdriver.Chrome()

link_list = []

for i in range(1,4):
    print("=" * 10,f"현재 {i} 페이지 입니다.", "=" * 10)
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)

    datas = browser.find_elements(By.CLASS_NAME, 'gd_name')

    for i in datas:
        link = i.get_attribute('href')

        link_list.append(link)
        # print(i.text)

    print(f"페이지가 끝났습니다.")

print(link_list)


========== 현재 1 페이지 입니다. ==========
페이지가 끝났습니다.
========== 현재 2 페이지 입니다. ==========
페이지가 끝났습니다.
========== 현재 3 페이지 입니다. ==========
페이지가 끝났습니다.
['https://www.yes24.com/Product/Goods/125557465', 'https://www.yes24.com/Product/Goods/128266166', 'https://www.yes24.com/Product/Goods/124999476', 'https://www.yes24.com/Product/Goods/123930880', 'https://www.yes24.com/Product/Goods/124225606', 'https://www.yes24.com/Product/Goods/127094902', 'https://www.yes24.com/Product/Goods/128133897', 'https://www.yes24.com/Product/Goods/104663596', 'https://www.yes24.com/Product/Goods/123930891', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/124043812', 'https://www.yes24.com/Product/Goods/124043802', 'https://www.yes24.com/Product/Goods/126511412', 'https://www.yes24.com/Product/Goods/126845471', 'https://www.yes24.com/Product/Goods/126702038', 'https://www.yes24.com/Product/Goods/125295101', 'https://www.yes24.com/Product/Goods/126516935', 'https://www.yes24.com

In [123]:
# for link in link_list:
#     browser.get(link)

browser.get(link_list[0])

title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(' ')[2]
price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best').find_element(By.TAG_NAME, 'a').text.split(' ')[-1].split('위')[0]
ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(' | ')[1].split(' ')[2].split('주')[0]

ranking, ranking_weeks

('1', '1')

In [1]:
#데이터 베이스 연동 후 -> DB에 저장
import pymysql
import pymysql.cursors
import time
import re
from datetime import datetime
connection = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = '1025',
    db = 'yes24',
    charset= 'utf8mb4',
    cursorclass= pymysql.cursors.DictCursor
)

with connection.cursor() as cursor:
    
    for link in link_list:

        browser.get(link)

        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text

        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일',  publishing)
        if match:
            year, month, day = match.groups()
            date_obj = datetime(int(year), int(month), int(day))
            publishing = date_obj.strftime("%Y-%m-%d")
        else:
            publishing = '2024-01-01'

        rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
        review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
        review = int(review.replace(",", ""))

        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(' ')[2]
        sales = int(sales.replace(",", ""))

        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(",", ""))
        try:
            ranking = browser.find_element(By.CLASS_NAME, 'gd_best').find_element(By.TAG_NAME, 'a').text.split(' ')[-1].split('위')[0]
            ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(' | ')[1].split(' ')[2].split('주')[0]
            ranking = int(ranking)
            ranking_weeks = int(ranking_weeks)
        except:
            ranking = 0
            ranking_weeks = 0

        sql = """
        INSERT INTO Books(title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        cursor.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        connection.commit()
        time.sleep(0.2)



NameError: name 'link_list' is not defined

In [ ]:
#상세 페이지 이동 후 데이터 크롤링